# You Only Look Once (YOLO)

Este componente utiliza o modelo YOLO para classificação usando [Yolov4](https://pypi.org/project/yolov4/). <br>
Está é uma biblioteca que encapsula o modelo yolov4 com algumas variações, podendo utilizar o modelo completo, como também,a versão reduzida.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
dataset = '/tmp/data/coco.zip'
language = "português" #@param ["português","inglês"] {type:"string",multiple:false,label:"Linguagem",description:"Linguagem a ser utilizada como descrição dos objetos identificados."}

## Extração dos dados do arquivo .zip

In [ ]:
folder = dataset.split('.')[0]

!mkdir -p {folder}
!unzip -o {dataset} -d {folder}

In [ ]:
import os

images = os.listdir(f'{folder}')

images = [x for x in images if not x.startswith('.')]

## Classificação das imagens

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

In [ ]:
# Download classes
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-yolo/coco.names
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-yolo/cocopt.names

# Download weights
import gdown
gdown.download('https://drive.google.com/uc?id=1L-SO373Udc9tPz5yLkgti5IAXFboVhUt', 'yolov4.weights', quiet=False)

In [ ]:
# Read and format classes list
names = None

if language == 'português':
    file = open('cocopt.names', 'r')
    names = file.read().split('\n')
else:
    file = open('coco.names', 'r')
    names = file.read().split('\n')

In [ ]:
from yolov4.tf import YOLOv4

# Load the model
yolo = YOLOv4() 

# Configure classes
yolo.classes = "coco.names"

# Load weights
yolo.make_model()
yolo.load_weights("yolov4.weights", weights_type="yolo")

In [ ]:
# Classify images

result = {}

for img in images:
    
    ext = img.split('.')[-1]
    
    if ext in ['jpg', 'jpeg', 'png']:
    
        frame = cv2.imread(f'{folder}/{img}')

        bboxes = yolo.predict(frame)

        result[img] = []

        for box in bboxes:
            result[img].append({'class': names[int(box[4])], 'probability': box[5], 'coords': box[0:4]})

In [ ]:
# Almost commented
if False:
    frame = yolo.draw_bboxes(frame, bboxes)
    cv2.imshow('image', frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "names": names,
    "result": result
}

dump(artifacts, "/tmp/data/yolo.joblib")